In [32]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Steps 3-7: Analyse the Content Using Cloud Natural Language API

To use the Natural Language API we will import the required libraries.

In [34]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [35]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/drive/My Drive/tempp/key.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

/drive/My Drive/tempp/key.json


In [36]:
df = pd.read_excel("/drive/My Drive/tempp/CIL_NERandSentiment.xlsx")
df.shape

(38244, 15)

In [37]:
df.fillna('', inplace=True)

In [15]:
# dfx = df[['clean_text']].head(5)
df.head(3)

,Unnamed: 0,index1,date,title,text,length,clean_text,tfidf_uni,tfidf_tri,clusters1_uni,text_ne,text_ner,cln_text_sentiment,sentiment,cluster_text
0,0,27.0,2022-04-27,How Long Does it Take to Get a Divorce in Illi...,"Before 2016, Illinois law famously, or infamou...",9649,illinois law famously infamously included twoy...,"['illinois', 'state', 'people', 'year', 'time'...","['social media accounts', 'illinois law requir...",0,<zip object at 0x7f9c39d28f00>,"{'faultbased': 3, 'illinois courts service pub...",0.012970,Positive,About Chicago/IL
1,1,92.0,2022-02-10,Edley: Illinois Democrats try to engineer arti...,By Bill Edley\nThe new Illinois 2022-2030 cong...,5496,edley illinois congressional map dubious disti...,"['chicago', 'illinois', 'state', 'year', 'poin...","['democratic national convention', 'illinois c...",0,<zip object at 0x7f9c39f61a50>,"{'democratic party': 3, 'budzinski': 1, 'congr...",0.033746,Positive,About Chicago/IL
2,2,182.0,2022-02-10,North Chicago Real Estate Lawyer | Waukegan Fo...,Bookmark According to information collected by...,3004,bookmark collected local outlets eviction mora...,"['chicago', 'illinois', 'state', 'covid', 'cit...","['chicago cook county', 'suburban cook county'...",0,<zip object at 0x7f9c3ac913c0>,"{'brett olmstead': 1, 'newland newland llp': 1...",0.024370,Positive,About Chicago/IL


In [38]:
cluster = 0
dfx = df[df['clusters1_uni'] == cluster]

In [41]:
dfx.head()

,Unnamed: 0,index1,date,title,text,length,clean_text,tfidf_uni,tfidf_tri,clusters1_uni,text_ne,text_ner,cln_text_sentiment,sentiment,cluster_text
0,0,27.0,2022-04-27,How Long Does it Take to Get a Divorce in Illi...,"Before 2016, Illinois law famously, or infamou...",9649,illinois law famously infamously included twoy...,"['illinois', 'state', 'people', 'year', 'time'...","['social media accounts', 'illinois law requir...",0,<zip object at 0x7f9c39d28f00>,"{'faultbased': 3, 'illinois courts service pub...",0.012970,Positive,About Chicago/IL
1,1,92.0,2022-02-10,Edley: Illinois Democrats try to engineer arti...,By Bill Edley\nThe new Illinois 2022-2030 cong...,5496,edley illinois congressional map dubious disti...,"['chicago', 'illinois', 'state', 'year', 'poin...","['democratic national convention', 'illinois c...",0,<zip object at 0x7f9c39f61a50>,"{'democratic party': 3, 'budzinski': 1, 'congr...",0.033746,Positive,About Chicago/IL
2,2,182.0,2022-02-10,North Chicago Real Estate Lawyer | Waukegan Fo...,Bookmark According to information collected by...,3004,bookmark collected local outlets eviction mora...,"['chicago', 'illinois', 'state', 'covid', 'cit...","['chicago cook county', 'suburban cook county'...",0,<zip object at 0x7f9c3ac913c0>,"{'brett olmstead': 1, 'newland newland llp': 1...",0.024370,Positive,About Chicago/IL
3,3,199.0,2022-04-28,New Chicago IG Asked to Investigate City Clerk...,"\nFor the first time in months, the city of Ch...",2869,time months city chicago inspector general deb...,"['chicago', 'illinois', 'state', 'company', 'c...","['mayor lori lightfoot', 'chicago city council...",0,<zip object at 0x7f9c39e3fa50>,"{'anna valencia': 3, 'nbc': 2, 'alexi giannoul...",-0.004444,Negative,About Chicago/IL
4,4,225.0,2022-01-12,Senate passes bill to honor Emmett Till and hi...,WASHINGTON — The Senate has passed a bill to a...,1961,washington — senate passed award congressional...,"['chicago', 'covid', 'time', 'students', 'scho...","['breaking follow stories', 'follow stories de...",0,<zip object at 0x7f9c39e03500>,"{'mamie tillmobley': 2, 'bobby rush': 1, 'bobb...",0.016116,Positive,About Chicago/IL


In [21]:
# df_secondrun = df_secondrun[(df_secondrun['length'] > 1500) & (df_secondrun['length'] < 10000)]


In [42]:
# df_t= df.head(1500)
# del df_t[df_t.columns[0]]
dfx.fillna('', inplace=True)
dfx.shape

(2000, 15)

In [43]:
# def google_nl_api(text_content):
from datetime import datetime
import time

dfx['g_sent_score']=""
dfx['illinois_entity_sentiment'] = ""
dfx['chicago_entity_sentiment'] = ""
dfx['g_category'] = ""
dfx['g_category_confidence'] = ""


client = language_v1.LanguageServiceClient()
encoding_type = enums.EncodingType.UTF8
type_ = enums.Document.Type.PLAIN_TEXT
language = "en"

start = datetime.now()
time.sleep(1);
counter = 0

for idx,row in dfx.iterrows():

  # while (True):

  #   end = datetime.now()
  #   s = (end-start).seconds
  #   if (counter >= 600):
  #       if (s <= 1):
  #           time.sleep(61) # You can keep track of the time and sleep less, actually
  #           start = datetime.now()
  #           print(f"Just Before Reset:{counter}")
  #           counter = 0

  text_content = row['text'].lower()
  document = {"content": text_content, "type": type_, "language": language}

  # Get sentiment annontations from the document
  annotations = client.analyze_sentiment(document, encoding_type=encoding_type)
  # Get overall document sentiment score
  dfx.loc[idx,'g_sent_score'] = annotations.document_sentiment.score


  #entity_sentiment
  aes_response = client.analyze_entity_sentiment(document, encoding_type=encoding_type)
  # Construct a list of entities where the entity type is a LOCATION
  entities = [entity for entity in aes_response.entities if enums.Entity.Type(entity.type).name == 'LOCATION' and entity.name in ['illinois','chicago']]
  # print(entities)


  
  # Loop through locations
  for entity in entities:
    score_dict =[]
    # print('\n' + '#'*50 + '\n')
    col_name = str(entity.name) + '_entity_sentiment'
    # print(col_name)
    sent = entity.sentiment
    score_dict.append(sent.score)
    score_dict.append(sent.magnitude)
    # score_dict['magnitude'] = sent.magnitude
    dfx.loc[idx,col_name] = str(score_dict)


  # categ ={}
  # category = client.classify_text(document)
  # # print(category.name)
  # # print(category.confidence)
  # # for category in response.categories:
  # #   categ[category.name] = category.confidence
  # #   break

  # # print(categ)
  # dfx.loc[idx,'g_category'] = category.name
  # dfx.loc[idx,'g_category_confidence'] = category.confidence

  end = datetime.now()
  print((end-start).seconds)
  print(f"Normal:{counter}")
  counter += 1

3
Normal:0
4
Normal:1
5
Normal:2
6
Normal:3
7
Normal:4
8
Normal:5
10
Normal:6
11
Normal:7
12
Normal:8
13
Normal:9
14
Normal:10
14
Normal:11
15
Normal:12
17
Normal:13
17
Normal:14
17
Normal:15
18
Normal:16
19
Normal:17
21
Normal:18
21
Normal:19
22
Normal:20
23
Normal:21
25
Normal:22
26
Normal:23
27
Normal:24
28
Normal:25
29
Normal:26
30
Normal:27
31
Normal:28
33
Normal:29
33
Normal:30
34
Normal:31
35
Normal:32
37
Normal:33
37
Normal:34
38
Normal:35
39
Normal:36
41
Normal:37
41
Normal:38
42
Normal:39
44
Normal:40
48
Normal:41
49
Normal:42
50
Normal:43
52
Normal:44
52
Normal:45
53
Normal:46
54
Normal:47
54
Normal:48
56
Normal:49
57
Normal:50
59
Normal:51
59
Normal:52
60
Normal:53
61
Normal:54
61
Normal:55
63
Normal:56
63
Normal:57
63
Normal:58
64
Normal:59
64
Normal:60
65
Normal:61
66
Normal:62
67
Normal:63
67
Normal:64
67
Normal:65
67
Normal:66
69
Normal:67
69
Normal:68
70
Normal:69
71
Normal:70
72
Normal:71
73
Normal:72
73
Normal:73
74
Normal:74
75
Normal:75
75
Normal:76
76
Normal:77
79

In [ ]:
# for idx, row in df_secondrun.iterrows():
#   df_secondrun.loc[idx,'google_sentiment'] = article_sentiments(row['clean_text'])

In [44]:
dfx.to_excel("/drive/My Drive/tempp/google_api_df_0.xlsx")

In [31]:
dfx.head()

,Unnamed: 0,index1,date,title,text,length,clean_text,tfidf_uni,tfidf_tri,clusters1_uni,text_ne,text_ner,cln_text_sentiment,sentiment,cluster_text,g_sent_score,illinois_entity_sentiment,chicago_entity_sentiment,g_category,g_category_confidence
0,0,27.0,2022-04-27,How Long Does it Take to Get a Divorce in Illi...,"Before 2016, Illinois law famously, or infamou...",9649,illinois law famously infamously included twoy...,"['illinois', 'state', 'people', 'year', 'time'...","['social media accounts', 'illinois law requir...",0,<zip object at 0x7f9c39d28f00>,"{'faultbased': 3, 'illinois courts service pub...",0.012970,Positive,About Chicago/IL,-0.2,"[0.0, 1.2000000476837158]",,,
1,1,92.0,2022-02-10,Edley: Illinois Democrats try to engineer arti...,By Bill Edley\nThe new Illinois 2022-2030 cong...,5496,edley illinois congressional map dubious disti...,"['chicago', 'illinois', 'state', 'year', 'poin...","['democratic national convention', 'illinois c...",0,<zip object at 0x7f9c39f61a50>,"{'democratic party': 3, 'budzinski': 1, 'congr...",0.033746,Positive,About Chicago/IL,-0.2,"[0.0, 0.8999999761581421]","[0.0, 0.5]",,
2,2,182.0,2022-02-10,North Chicago Real Estate Lawyer | Waukegan Fo...,Bookmark According to information collected by...,3004,bookmark collected local outlets eviction mora...,"['chicago', 'illinois', 'state', 'covid', 'cit...","['chicago cook county', 'suburban cook county'...",0,<zip object at 0x7f9c3ac913c0>,"{'brett olmstead': 1, 'newland newland llp': 1...",0.024370,Positive,About Chicago/IL,-0.1,"[0.0, 0.10000000149011612]","[0.0, 0.20000000298023224]",,
3,3,199.0,2022-04-28,New Chicago IG Asked to Investigate City Clerk...,"\nFor the first time in months, the city of Ch...",2869,time months city chicago inspector general deb...,"['chicago', 'illinois', 'state', 'company', 'c...","['mayor lori lightfoot', 'chicago city council...",0,<zip object at 0x7f9c39e3fa50>,"{'anna valencia': 3, 'nbc': 2, 'alexi giannoul...",-0.004444,Negative,About Chicago/IL,-0.2,"[0.0, 0.0]","[0.0, 0.20000000298023224]",,
4,4,225.0,2022-01-12,Senate passes bill to honor Emmett Till and hi...,WASHINGTON — The Senate has passed a bill to a...,1961,washington — senate passed award congressional...,"['chicago', 'covid', 'time', 'students', 'scho...","['breaking follow stories', 'follow stories de...",0,<zip object at 0x7f9c39e03500>,"{'mamie tillmobley': 2, 'bobby rush': 1, 'bobb...",0.016116,Positive,About Chicago/IL,-0.1,,"[0.0, 0.0]",,


## Test cases

In [9]:
articles = df['clean_text'].tolist()
articles

['illinois law famously infamously included twoyear waiting period people wait fortyeight months nofault divorce workarounds spouses file faultbased divorces citing issues cruel treatment faultbased divorces messy subject waiting period spouses wait longer wait illinois divorce laws changed emotional financial issues divorce case ’ alter divorce settlements altered divorces started lawmakers ended faultbased divorces eliminated twoyear nofault divorce waiting period change sweeping sounds divorce procedure unaltered change speed things bit today ’ impossible determine average length divorce illinois google phrase number average length divorce average length marriage couples ’ anniversaries buried decades wedded bliss divorce final blink eye drag years naperville divorce lawyer long case inexperienced telling hear naperville divorce lawyer guide process resolution upholds legal financial rights ’ divorce issues number issues divorce contentiousness issues greatly affect length marriage 

In [33]:
language = "en"
type_ = enums.Document.Type.PLAIN_TEXT


# Analyse the latest 5 articles 
for article in articles[:3]:
  # print (article)
  print('\n' + '#'*50 + '\n')
  # print(article["title"])
  # print(article["link"])
  document = {"content": article, "type": type_, "language": language}
  print_sentiments(document)  
print('\n' + '#'*50)


##################################################

Overall sentiment: NEGATIVE
Entity: illinois
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.5

##################################################

Overall sentiment: NEGATIVE
Entity: illinois
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.4000000059604645
Entity: chicago
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0

##################################################

Overall sentiment: NEGATIVE
Entity: chicago
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Entity: illinois
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0

##################################################


In [49]:

def sample_classify_text(text_content):
    """
    Classifying Content in a String

    Args:
      text_content The text content to analyze. Must include at least 20 words.
    """

    client = language_v1.LanguageServiceClient()
    encoding_type = enums.EncodingType.UTF8

    # text_content = 'That actor on TV makes movies in Hollywood and also stars in a variety of popular new TV shows.'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    response = client.classify_text(document)
    # Loop through classified categories returned from the API
    for category in response.categories:

        print(u"Category name: {}".format(category.name))
        # Get the confidence. Number representing how certain the classifier
        print(u"Confidence: {}".format(category.confidence))


In [50]:
# Analyse the latest 5 articles 
for article in articles:
  print('\n' + '#'*50 + '\n')
  sample_classify_text(article)
 
print('\n' + '#'*50)


##################################################

Category name: /Law & Government/Legal
Confidence: 0.9800000190734863

##################################################

Category name: /News/Politics
Confidence: 0.9399999976158142
Category name: /Law & Government/Government
Confidence: 0.5799999833106995

##################################################

Category name: /Real Estate
Confidence: 0.75
Category name: /Law & Government/Legal/Legal Services
Confidence: 0.5199999809265137

##################################################

Category name: /Law & Government/Government
Confidence: 0.6200000047683716

##################################################

Category name: /News/Politics
Confidence: 0.9100000262260437
Category name: /Law & Government/Government
Confidence: 0.5699999928474426

##################################################


In [34]:
articles[:3]

['illinois law famously infamously included twoyear waiting period people wait fortyeight months nofault divorce workarounds spouses file faultbased divorces citing issues cruel treatment faultbased divorces messy subject waiting period spouses wait longer wait illinois divorce laws changed emotional financial issues divorce case ’ alter divorce settlements altered divorces started lawmakers ended faultbased divorces eliminated twoyear nofault divorce waiting period change sweeping sounds divorce procedure unaltered change speed things bit today ’ impossible determine average length divorce illinois google phrase number average length divorce average length marriage couples ’ anniversaries buried decades wedded bliss divorce final blink eye drag years naperville divorce lawyer long case inexperienced telling hear naperville divorce lawyer guide process resolution upholds legal financial rights ’ divorce issues number issues divorce contentiousness issues greatly affect length marriage 